In [ ]:
DATA_URL_2025_10 = (
    "https://www.comunidad.madrid/sites/default/files/doc/sanidad/gest/le_c_octubre_2025.csv"
)

In [ ]:
import requests
import pandas as pd

In [ ]:
raw_data = requests.get(DATA_URL_2025_10)
raw_data

In [ ]:
raw_data.encoding

In [ ]:
raw_data.text

In [ ]:
df_oct25 = pd.read_csv(
    DATA_URL_2025_10, encoding=raw_data.encoding, sep=";", header=2, skipfooter=2, engine="python"
)
df_oct25.head()

In [ ]:
df_oct25.tail()

In [ ]:
df_oct25_pre = df_oct25.set_index("OCTUBRE 2025")
df_oct25_pre["Comunidad de Madrid"] = df_oct25_pre["Comunidad de Madrid"].str.replace(".", "").str.replace(",", ".").astype(float)
df_oct25_pre.columns = ["2025-10"]
df_oct25_pre.index.name = None

df_oct25_pre.head()

In [ ]:
dfs = [
    df_oct25_pre,  # ...
]

In [ ]:
df = pd.concat(dfs, axis="columns").T
df

In [ ]:
BASE_URL = "https://www.comunidad.madrid/sites/default/files/doc/sanidad/gest/"

def get_url(hc_type: str, year: int, month: int) -> str:
    type_particles = {
        "outpatient": "c",
        "diagnostic": "t",
        "surgery": "q",
    }
    month_names = {
        1: "enero",
        2: "febrero",
        3: "marzo",
        4: "abril",
        5: "mayo",
        6: "junio",
        7: "julio",
        8: "agosto",
        9: "septiembre",
        10: "octubre",
        11: "noviembre",
        12: "diciembre",
    }
    return f"{BASE_URL}le_{type_particles[hc_type]}_{month_names[month]}_{year}.csv"

get_url("outpatient", 2025, 10)

In [ ]:
def raw_from_url(url: str, encoding: str):
    return pd.read_csv(
        url, encoding=encoding, sep=";", header=2, skipfooter=2, engine="python"
    )

In [ ]:
def read_data(hc_type: str, year: int, month: int):
    url = get_url(hc_type, year, month)
    resp = requests.get(url)
    if resp.status_code != 200:
        return

    df_raw = raw_from_url(url, resp.encoding)

    df_pre = df_raw.set_index(df_raw.columns[0])
    df_pre.iloc[:, 0] = df_pre.iloc[:, 0].str.replace(".", "").str.replace(",", ".").astype(float)
    df_pre.columns = [f"{year}-{month:02d}"]
    df_pre.index.name = None
    return df_pre

In [ ]:
read_data("outpatient", 2025, 1)

In [ ]:
def assemble_data(hc_type: str, year: int):
    all_dfs = []
    for month in range(1, 13):
        df = read_data(hc_type, year, month)
        if df is not None:
            all_dfs.append(df)

    return pd.concat(all_dfs, axis="columns").T

In [ ]:
assemble_data("outpatient", 2025)